## Nafisa's Refugee Piece
http://journalism.frontanalytics.com/2017/11/28/the-utah-refugees/  
https://github.com/nafisamasud/utah_refugees

In [1]:
import pandas as pd
import random
import csv

In [2]:
df = pd.read_csv('utah_refugees/data/raw_data.csv')
print(df.shape)

(12486, 8)


In [3]:
# There are quite a few (~104 null rows)
df[df.isnull().any(axis=1)].head()

,FY of Arrival,CY of Arrival,CY of birth,Age at Arrival,Gender,Country of Birth,Arrival From,Nativity/Culture
833,2017,2017,1997,20,NaN,SY,JO,SY
1197,2016,2016,1984,32,NaN,CT,DC,CT
3679,2014,2014,1989,24,M,BM,NaN,BM
4679,2014,2014,1981,33,M,SU,NaN,SU
4706,2014,2014,1992,22,M,SU,NaN,SU


In [4]:
# Drop the duplicates
df = df.dropna()

In [5]:
df.shape

(12385, 8)

In [6]:
# read in the lookup table
lookup_df = pd.read_csv('utah_refugees/data/lookup_table.csv')

In [7]:
lookup_dict = lookup_df.to_dict(orient='index')

In [8]:
new_lookup = {}
for k, v in lookup_dict.items():
    nk = v['Code']
    nd = v['Description']
    new_lookup[nk] = nd

In [9]:
# why is this in there?
del new_lookup['FY']

In [10]:
df.head()

,FY of Arrival,CY of Arrival,CY of birth,Age at Arrival,Gender,Country of Birth,Arrival From,Nativity/Culture
0,2017,2017,2010,7,M,AF,AF,AF
1,2017,2017,1986,31,M,AF,AF,AF
2,2017,2017,1987,30,F,AF,AF,AF
3,2017,2017,2012,5,F,AF,AF,AF
4,2017,2017,1984,33,M,AF,AF,AF


In [11]:
# Map on longer descriptions of countries
df['Country of Birth_long'] = df['Country of Birth'].map(new_lookup)
df['Arrival From_long'] = df['Arrival From'].map(new_lookup)
df['Nativity/Culture_long'] = df['Nativity/Culture'].map(new_lookup)

In [12]:
df.head()

,FY of Arrival,CY of Arrival,CY of birth,Age at Arrival,Gender,Country of Birth,Arrival From,Nativity/Culture,Country of Birth_long,Arrival From_long,Nativity/Culture_long
0,2017,2017,2010,7,M,AF,AF,AF,Afghanistan,Afghanistan,Afghanistan
1,2017,2017,1986,31,M,AF,AF,AF,Afghanistan,Afghanistan,Afghanistan
2,2017,2017,1987,30,F,AF,AF,AF,Afghanistan,Afghanistan,Afghanistan
3,2017,2017,2012,5,F,AF,AF,AF,Afghanistan,Afghanistan,Afghanistan
4,2017,2017,1984,33,M,AF,AF,AF,Afghanistan,Afghanistan,Afghanistan


#### Calculate the percentage of all immigrants, by year, by country, by gender

In [13]:
# Group it so we have a by year, by gender, by country df
yr_cnt = df.groupby(['CY of Arrival', 'Gender', 'Country of Birth'])['Nativity/Culture'].agg('count')
final = yr_cnt.groupby(level=[0, 1]).apply(lambda x: x / float(x.sum()))

In [14]:
# Get a unique set of all used countries in the dataset
all_countries = set(df['Country of Birth'].tolist() + df['Arrival From'].tolist() 
                    + df['Nativity/Culture'].tolist())

In [15]:
final_df = pd.DataFrame(final)
groups = final_df.groupby(['CY of Arrival', 'Gender'])

#### Adding in missing countries

In [16]:
# Not all years, genders have all the countries. If we don't have them all, the viz breaks
# Here we add in all the missing countries to each group
output = []
cols = ['CY of Arrival', 'Gender', 'Country of Birth', 'Nativity/Culture']
for _, group in groups:
    year = _[0]
    gender = _[1]
    g = group.reset_index()
    these_countries = g['Country of Birth'].tolist()
    missing = [c for c in all_countries if c not in these_countries]
    blanks = pd.concat([pd.DataFrame([{'CY of Arrival': year, 'Gender': gender,
                   'Country of Birth': i, 'Nativity/Culture': 0.0}]) for i in missing], ignore_index=True)
    g = pd.concat([g, blanks])
    output.append(g.reset_index())

In [17]:
# Compile all the groups
lengthend_df = pd.concat(output)

In [18]:
lengthend_df.head()

,index,CY of Arrival,Country of Birth,Gender,Nativity/Culture
0,0,2006,AJ,F,0.018182
1,1,2006,BM,F,0.054545
2,2,2006,CU,F,0.200000
3,3,2006,HA,F,0.036364
4,4,2006,ID,F,0.018182


In [19]:
lengthend_df = lengthend_df.drop('index', axis=1)

In [20]:
# reset the columns for something easier to use with D3
lengthend_df.columns = ['yoa', 'country', 'gender', 'perc']

In [21]:
# Test that year/gender combos adds up to 1
lengthend_df[(lengthend_df['yoa'] == 2011) & (lengthend_df['gender'] == 'F')]['perc'].sum()

1.0

In [22]:
# Round out the percents
lengthend_df['perc'] = lengthend_df['perc'].apply(lambda x: round(x, 4))

In [23]:
lengthend_df.head()

,yoa,country,gender,perc
0,2006,AJ,F,0.0182
1,2006,BM,F,0.0545
2,2006,CU,F,0.2000
3,2006,HA,F,0.0364
4,2006,ID,F,0.0182


#### Find countries that are missing in the lookup table

In [24]:
# For now we just give them random year within that range for where the label will go
# Once we view the graph, we can update it by trial and error
updated_lookup = {}
for k, v in new_lookup.items():
    yr = random.choice(seq=[i for i in range(2007, 2017)])
    updated_lookup[k] = {'descrip': '', 'name': v, 'xlab': yr}

After some trial and error...

In [25]:
# these are countries that have enough room for labels in the visualization
# Only discovered after lots of testing
label_countries = [
    'Burma', 'Thailand', 'Iran', 'Bhutan', 'Iraq', 'Somalia', 'Afghanistan', 'Dem Republic of the Congo',
    'Tanzania', 'Congo'
]
countries_with_positions = {'AF': {'include': 1, 'year': 2016},
 'BM': {'include': 1, 'year': 2010},
 'BT': {'include': 1, 'year': 2010},
 'CG': {'include': 1, 'year': 2013},
 'CU': {'include': 1, 'year': 2016},
 'DC': {'include': 1, 'year': 2016},
 'ET': {'include': 1, 'year': 2015},
 'IR': {'include': 1, 'year': 2007},
 'IZ': {'include': 1, 'year': 2013},
 'KE': {'include': 1, 'year': 2009},
 'NP': {'include': 1, 'year': 2012},
 'RW': {'include': 1, 'year': 2016},
 'SO': {'include': 1, 'year': 2012},
 'TH': {'include': 1, 'year': 2011},
 'TZ': {'include': 1, 'year': 2016},
 'UG': {'include': 1, 'year': 2016}}

In [26]:
# so we can update these values here
updated_updated_lookup = {}
for k, v in updated_lookup.items():
    if k in countries_with_positions.keys():
        v['include'] = 1
        v['xlab'] = countries_with_positions.get(k)['year']
    else:
        v['include'] = 0
    updated_updated_lookup[k] = v

In [27]:
updated_updated_lookup['KE']

{'descrip': '', 'include': 1, 'name': 'Kenya', 'xlab': 2009}

^^^^ This dictionary gets referenced in the `refugees.html` script as the `var places_meta` variable.

Find countries that aren't in the lookup table

In [28]:
# Countries with data
current_countries = set(lengthend_df['country'].tolist())

In [29]:
# Countries available in the lookup table
lookup_values = list(updated_updated_lookup.keys())

In [30]:
# Which countries are not included in the lookup table
# we'll have to filter them out
[c for c in current_countries if c not in lookup_values]

['GM', 'IT']

Filter out missing countries

In [31]:
# Filter out missing countries
lengthend_df = lengthend_df[~lengthend_df['country'].isin(['GM', 'IT'])]

Sort the values so the colors match up by country and gender

In [32]:
# Important step - sort so colors match up
finaldf = lengthend_df.sort_values(['yoa', 'country', 'gender']).reset_index(drop=True)

In [34]:
finaldf.to_csv('utah_refugees/visuals/countries.tsv', index=False, sep='\t')